### Import thư viện và Cấu hình thiết bị

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
import time
import sys
import nltk
from nltk.translate.bleu_score import corpus_bleu

# Import các module tự định nghĩa (đảm bảo file data_utils.py và model.py nằm cùng thư mục)
from data_utils import Multi30kDataset, Collate, tokenize_en
from model import Encoder, Decoder, Seq2Seq

# Cấu hình thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔹 Đang sử dụng thiết bị: {device}")

# Cấu hình đường dẫn (Lưu ý: Notebook không có __file__, dùng os.getcwd())
current_dir = os.getcwd() 
data_dir = os.path.join(current_dir, 'data', 'multi30k_en_fr')

# Đảm bảo thư mục tồn tại
if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)
    print(f"⚠️ Thư mục dữ liệu chưa tồn tại: {data_dir}. Hãy đảm bảo bạn đã tải dữ liệu.")

train_src = os.path.join(data_dir, 'train.en')
train_trg = os.path.join(data_dir, 'train.fr')
val_src = os.path.join(data_dir, 'val.en') 
val_trg = os.path.join(data_dir, 'val.fr')

🔹 Đang sử dụng thiết bị: cuda


### Khởi tạo Dataset và DataLoader

In [9]:
print("1. KHỞI TẠO DỮ LIỆU...")

# Load dataset
# Lưu ý: Nếu file chưa tồn tại, Dataset có thể báo lỗi tùy vào cách bạn viết data_utils
train_dataset = Multi30kDataset(train_src, train_trg)

# Dùng chung vocab cho validation nếu có
if os.path.exists(val_src):
    val_dataset = Multi30kDataset(val_src, val_trg, src_vocab=train_dataset.src_vocab, trg_vocab=train_dataset.trg_vocab)
else:
    print("⚠️ Không tìm thấy dữ liệu Validation, dùng tạm tập Train để test code.")
    val_dataset = train_dataset

BATCH_SIZE = 128

# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          collate_fn=Collate(train_dataset.src_vocab.stoi["<pad>"]),
                          num_workers=0, pin_memory=True)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        collate_fn=Collate(train_dataset.src_vocab.stoi["<pad>"]),
                        num_workers=0, pin_memory=True)

print(f"✅ Đã load dữ liệu. Vocab nguồn: {len(train_dataset.src_vocab)}, Vocab đích: {len(train_dataset.trg_vocab)}")

1. KHỞI TẠO DỮ LIỆU...
🔹 Đang đọc: train.en...
🔹 Đang đọc: val.en...
✅ Đã load dữ liệu. Vocab nguồn: 6191, Vocab đích: 6555


### Khởi tạo Model, Optimizer và Loss Function

In [10]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

print("2. KHỞI TẠO MODEL...")

INPUT_DIM = len(train_dataset.src_vocab)
OUTPUT_DIM = len(train_dataset.trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
EPOCHS = 16

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)
model.apply(init_weights)

# Đếm số lượng tham số
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.trg_vocab.stoi["<pad>"])
scaler = torch.amp.GradScaler('cuda')

2. KHỞI TẠO MODEL...
The model has 13,982,107 trainable parameters


### Định nghĩa hàm Train và Translate nội bộ

In [11]:
def train_epoch(model, iterator, optimizer, criterion, clip, scaler, device):
    model.train()
    epoch_loss = 0
    for i, (src, trg, src_len) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        # Mixed Precision Training
        with torch.amp.autocast('cuda'):
            output = model(src, src_len, trg)
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
        
        if i % 50 == 0:
            print(f"   Batch {i}/{len(iterator)} | Loss: {loss.item():.4f}")
            
    return epoch_loss / len(iterator)

def translate_sentence_internal(sentence, src_vocab, trg_vocab, model, device, max_len=50):
    """Hàm dịch dùng để tính BLEU (trả về list tokens)"""
    model.eval()
    if isinstance(sentence, str):
        tokens = tokenize_en(sentence)
    else:
        tokens = sentence
    tokens = ["<sos>"] + tokens + ["<eos>"]
    src_indexes = src_vocab.numericalize(tokens)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor, src_len)
        
    trg_indexes = [trg_vocab.stoi["<sos>"]]
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_vocab.stoi["<eos>"]:
            break
            
    trg_tokens = [trg_vocab.itos[i] for i in trg_indexes]
    return trg_tokens[1:-1]

def calculate_bleu(data, src_vocab, trg_vocab, model, device):
    targets = []
    outputs = []
    print("   Đang tính BLEU trên 200 mẫu đầu tiên...")
    # Giới hạn 200 mẫu để tiết kiệm thời gian
    for i in range(min(len(data), 200)):
        src = data.src_tokenized[i]
        trg = data.trg_tokenized[i]
        
        pred = translate_sentence_internal(src, src_vocab, trg_vocab, model, device)
        
        targets.append([trg])
        outputs.append(pred)
        
    return corpus_bleu(targets, outputs)

### Vòng lặp Huấn luyện

In [13]:
import os # Import lại cho chắc chắn nếu chạy độc lập

print("4. ĐÁNH GIÁ KẾT QUẢ")

# Định nghĩa lại đường dẫn file model (để không phụ thuộc vào cell trên)
if 'current_dir' not in locals():
    current_dir = os.getcwd()
save_path = os.path.join(current_dir, 'best-model.pth')

# Load lại model tốt nhất
if os.path.exists(save_path):
    model.load_state_dict(torch.load(save_path, map_location=device)) # Thêm map_location để tránh lỗi nếu load trên CPU
    print(f"✅ Đã load model tốt nhất từ: {save_path}")
else:
    print(f"⚠️ Không tìm thấy file '{save_path}'. Đang sử dụng model hiện tại (chưa load lại).")

# Tính toán BLEU
score = calculate_bleu(val_dataset, train_dataset.src_vocab, train_dataset.trg_vocab, model, device)
print(f'🏆 BLEU Score = {score*100:.2f}')

4. ĐÁNH GIÁ KẾT QUẢ
✅ Đã load model tốt nhất từ: d:\LTSM\best-model.pth
   Đang tính BLEU trên 200 mẫu đầu tiên...


C:\Users\GIA HUY\AppData\Local\Temp\ipykernel_26236\1461541616.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path, map_location=

🏆 BLEU Score = 30.42


### Hàm Dịch (Inference) và Demo

In [14]:
def translate(sentence, model, src_vocab, trg_vocab, device):
    """Hàm dịch chuỗi string đầu vào sang tiếng Pháp"""
    
    # 1. Tokenize
    if isinstance(sentence, str):
        tokens = tokenize_en(sentence)
    else:
        tokens = sentence 

    # Thêm <sos>, <eos>
    tokens = ["<sos>"] + tokens + ["<eos>"]
    
    # Chuyển thành chỉ số 
    src_indexes = src_vocab.numericalize(tokens)
    
    # Chuyển thành tensor và thêm chiều batch
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    src_len = torch.LongTensor([len(src_indexes)])

    # 2. Encoder
    model.eval()
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor, src_len)

    # 3. Decoder
    trg_indexes = [trg_vocab.stoi["<sos>"]]
    max_len = 50 
    
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        
        # Greedy decoding
        pred_token = output.argmax(1).item()
        
        if pred_token == trg_vocab.stoi["<eos>"]:
            break
        
        trg_indexes.append(pred_token)

    # 4. Convert back to string
    trg_tokens = [trg_vocab.itos[i] for i in trg_indexes]
    
    # Bỏ <sos> ở đầu
    return " ".join(trg_tokens[1:])

# --- DEMO ---
examples = [
    "Two young, white males are outside near many bushes.",
    "A man in an orange hat starring at something.",
    "A group of people standing in front of an igloo."
]

print("--- KẾT QUẢ DỊCH THỬ ---")
for ex in examples:
    trans = translate(ex, model, train_dataset.src_vocab, train_dataset.trg_vocab, device)
    print(f"🇬🇧: {ex}")
    print(f"🇫🇷: {trans}")
    print("-" * 30)

--- KẾT QUẢ DỊCH THỬ ---
🇬🇧: Two young, white males are outside near many bushes.
🇫🇷: Deux jeunes hommes blancs sont dehors près de nombreux gens .
------------------------------
🇬🇧: A man in an orange hat starring at something.
🇫🇷: Un homme avec un casque orange regardant quelque chose .
------------------------------
🇬🇧: A group of people standing in front of an igloo.
🇫🇷: Un groupe de personnes debout devant une maison .
------------------------------


### Giao diện Dịch tương tác

In [ ]:
# Chạy cell này để tự nhập câu
while True:
    try:
        sentence = input("\nNhập câu tiếng Anh (gõ 'q' để thoát): ")
        if sentence.lower() in ['q', 'quit']: 
            print("Đã thoát.")
            break
        
        result = translate(sentence, model, train_dataset.src_vocab, train_dataset.trg_vocab, device)
        print(f"🇫🇷 French:  {result}")
    except Exception as e: 
        print(f"Lỗi: {e}")